# Optimisation Test Notebook - Insertion Heuristic

This notebook is created so the dynamic optimisation using the insertion heuristic can be performed.

In [ ]:
import os

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import time

from plotly.subplots import make_subplots

import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import interact
from copy import deepcopy

### 1. Run the model

Run the model by applying the requests dataset generated in the Request generation test notebook.

In [2]:
import Main_insertion

In [3]:
# get the file names of the excel files in Results_requests folder
files = os.listdir('../Results_requests')
# only the requests files
files = [file for file in files if 'Requests_' in file]

# let the user choose the file
# file can be distiguished by vessel type, demand level, and the instance number
# the user can choose the file by the vessel type, demand level, and the instance number

vessel_types = {'Parcel': 0 , 'Passenger': 1, 'Mixed': 2}
# flip the keys and values
vessel_types_inv = dict((v,k) for k,v in vessel_types.items())
demand_levels = ['low', 'high']
# depends on the max value of the instance number
N_inst = len(files) / (2*7)

demand_levels = input('Enter the demand level (low, high): ')
instance_number = input(f'Enter the instance number (0-{int(N_inst-1)}): ')

# ask if all patterns should be run or just one
Isall = input('Do you want to run all patterns? (yes, no): ')

if Isall == 'no':
    # let the user choose - type your input
    vessel_type_0 = input('Enter the vessel type for vessel 0 (Parcel, Passenger, Mixed): ')
    vessel_type_1 = input('Enter the vessel type for vessel 1 (Parcel, Passenger, Mixed): ')

    try:
        vessel_type_0 = vessel_types[vessel_type_0]
        vessel_type_1 = vessel_types[vessel_type_1]
        high
        # get the file name
        filenames = [f'Requests_K0_{vessel_type_0}_K1_{vessel_type_1}_demand_{demand_levels}_instance_{instance_number}.xlsx']

        # Assert that the file exists
        assert filenames[0] in files, 'File does not exist, please check the input.'

    except KeyError:
        print('Invalid vessel type')
        exit()

elif Isall == 'yes':
    # create a list of all the filenames 
    filenames = [file for file in files if f'demand_{demand_levels}_instance_{instance_number}' in file]

    # Assert that the file exists
    assert len(filenames) > 0, 'File does not exist, please check the input.'




In [4]:
# output folder
if not os.path.exists('../Results_insertion'):
    os.makedirs('../Results_insertion')

output_folder = '../Results_insertion/'

In [ ]:
# read results if they exist
try:
    if Isall == 'no':
        dict_df_results[(vessel_type_0, vessel_type_1)] = pd.read_excel(f'{output_folder}output_insertion_{demand_levels}_[{vessel_type_0},{vessel_type_1}_{instance_number}.xlsx')
    elif Isall == 'yes':
        dict_df_results = {}
        vessseltypes = [(0,1), (0,2), (1,0), (1,2), (2,0), (2,1), (2,2)]
        for v in vessseltypes:
            dict_df_results[v] = pd.read_excel(f'{output_folder}output_insertion_{demand_levels}_[{v[0]}, {v[1]}]_{instance_number}.xlsx')

except:
    print('Results do not exist, please run the requests first.')


Results do not exist, please run the requests first.


In [ ]:
# show the filnames the model will run
print('The model will run the following files:')
for file in filenames:
    print(file)

Below, the model runs.

In [7]:
dict_df_results = {}
dict_figures = {}
for file in filenames:
    demand_levels = file.split('_')[6]
    instance_number = file.split('_')[8].split('.')[0]
    vessel_type_0 = int(file.split('_')[2])
    vessel_type_1 = int(file.split('_')[4])
    data_path = f'../Results_requests/{file}'
    print("Start running the insertion algorithm for the configuration: ")
    print(f'Vessel 0: {vessel_type_0}, Vessel 1: {vessel_type_1}, Demand level: {demand_levels}, Instance number: {instance_number}', "\n")
    start = time.time()
    df_results, fig_dict = Main_insertion.main(data_path, output_folder, scenario=demand_levels, instance=instance_number, pattern=[vessel_type_0, vessel_type_1])
    dict_df_results[(vessel_type_0, vessel_type_1)] = df_results
    dict_figures[(vessel_type_0, vessel_type_1)] = fig_dict
    end = time.time()
    print(f'Calculation time: {end-start}', "\n")


Start running the insertion algorithm for the configuration: 
Vessel 0: 0, Vessel 1: 1, Demand level: high, Instance number: 1 

Start the optimisation
Optimisation completed! 

Calculation time: 93.13214683532715 

Start running the insertion algorithm for the configuration: 
Vessel 0: 0, Vessel 1: 2, Demand level: high, Instance number: 1 

Start the optimisation
Optimisation completed! 

Calculation time: 51.28398609161377 

Start running the insertion algorithm for the configuration: 
Vessel 0: 1, Vessel 1: 0, Demand level: high, Instance number: 1 

Start the optimisation
Optimisation completed! 

Calculation time: 55.70129442214966 

Start running the insertion algorithm for the configuration: 
Vessel 0: 1, Vessel 1: 2, Demand level: high, Instance number: 1 

Start the optimisation
Optimisation completed! 

Calculation time: 87.85885882377625 

Start running the insertion algorithm for the configuration: 
Vessel 0: 2, Vessel 1: 0, Demand level: high, Instance number: 1 

Start t

### Looking into the results

After executing the optimisation with different vessel type patterns, we are interested in the transportation performance of each patterns. The calculated KPIs will be visualised to provide insights into the difference in performance between different vessel type patterns. 

In [32]:
# Visualise the final total distance for each pattern as a bar chart
# collect the final total distance for each pattern

total_distances = [dict_df_results[key].iloc[-1]['Total_dist'] for key in dict_df_results.keys()]
total_distances_k0 = [dict_df_results[key].iloc[-1]['Travel_distance_k0'] for key in dict_df_results.keys()]
total_distances_k1 = [dict_df_results[key].iloc[-1]['Travel_distance_k1'] for key in dict_df_results.keys()]

# collect the final met ratio for each pattern
met_ratios = [round(dict_df_results[key].iloc[-1]['Met_Ratio']*100, 2) for key in dict_df_results.keys()]


fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Bar(x=[f'{vessel_types_inv[int(key[0])]} - {vessel_types_inv[int(key[1])]}' for key in dict_df_results.keys()], y=total_distances, name='Total distance [km]'), secondary_y=False)
fig.add_trace(go.Bar(x=[f'{vessel_types_inv[int(key[0])]} - {vessel_types_inv[int(key[1])]}' for key in dict_df_results.keys()], y=total_distances_k0, name='Total distance vessel 0 [km]'), secondary_y=False)
fig.add_trace(go.Bar(x=[f'{vessel_types_inv[int(key[0])]} - {vessel_types_inv[int(key[1])]}' for key in dict_df_results.keys()], y=total_distances_k1, name='Total distance vessel 1 [km]'), secondary_y=False)
fig.update_traces(width=0.2)


fig.add_trace(go.Scatter(x=[f'{vessel_types_inv[int(key[0])]} - {vessel_types_inv[int(key[1])]}' for key in dict_df_results.keys()], y=met_ratios, mode='lines+markers', name='Request met ratio [%]'), secondary_y=True)

fig.update_yaxes(title_text="Total distance [km]", secondary_y=False)
fig.update_yaxes(title_text="Met ratio [%]", secondary_y=True)
fig.update_layout(title='Total distance and Request met ratio for each pattern', xaxis_title='Pattern', barmode='group')


fig.show()